# Predicting Loan Default: Part 2

**EDA**

In [ ]:
# importing libraries

import sweetviz as sw
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import lightgbm as lgb

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

import re

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
train_clean = pd.read_csv('data/train_clean.csv')

In [ ]:
train_clean.head()

Using SweetViz to get a general look at the data, saved as output.html

In [ ]:
# analyze data
analyze_report = sw.analyze(train_clean)
# display analyzed data in browser
analyze_report.show_html('output.html', open_browser = True)

Initial EDA saved to html output.

In [ ]:
# using .describe() to take a look at the data
train_clean.describe()

In [ ]:
# split into continuous and categorical dataframes
train_num = train_clean.select_dtypes(include ='number')
train_cat = train_clean.select_dtypes(include = ['object','category'])

train_cat

In [ ]:
train_num

In [ ]:
train_cat = pd.read_csv('data/train_cat.csv')
train_num = pd.read_csv('data/train_num.csv')

In [ ]:
train_cat.head()

**Categorical Features**

Taking a look at the categorical features and seeing the number of defaults as a percentage of the total number of those feature value observations.

In [ ]:
# batch_enrolled
train_clean.groupby(['batch_enrolled'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

A higher percentage of default was noticed in BAT1135695.

In [ ]:
# grade
train_clean.groupby(['grade'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

Generally even percentage of defaults across most grades, with small increments the higher the alphabet grade.

In [ ]:
# sub_grade
train_clean.groupby(['sub_grade'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

A higher percentage of defaults in sub grade G3 as compared to the rest of the sub grades. It is also noted that perhaps using sub grade as the feature and removing grade would be a good idea as they are overlapping features.

In [ ]:
# employment_duration
train_clean.groupby(['employment_duration'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

Generally even distribution in terms of percentage across all 3 values but the highest percentage noticed in OWN.

In [ ]:
# verification_status
train_clean.groupby(['verification_status'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

A generally even distribution of defaults across all 3 observation types. 

In [ ]:
# loan_title
train_clean.groupby(['loan_title'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

A higher default rate noticed in refi, wedding loan, home, car loan, loan and relief.

In [ ]:
# initial_list_status
train_clean.groupby(['initial_list_status'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

Not too clear a difference in the distribution of default loans between the 2 values in initial list status, but forwarded initial list status has slightly higher rates of default than waiting.

In [ ]:
# application_type
train_clean.groupby(['application_type'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

Again very little difference in the percentages of loan defauults between individual and joint applications, with individual being slightly higher in rate.

**Numerical Features**

In [ ]:
train_num.head()

In [ ]:
train_num = train_num.apply(pd.to_numeric, errors='coerce')

In [ ]:
train_num.dtypes

In [ ]:
train_num.info()

In [ ]:
train_num.describe()

**Histplots**

Plotting some of the continuous features against the target column to see if any patterns can be seen from the plots.

In [ ]:
sns.histplot(data = train_clean, x = 'funded_amount', hue = 'loan_status', legend = True, kde = False, element = 'poly', bins = 100)
plt.xticks(rotation = 90);

In [ ]:
sns.histplot(data = train_clean, x = 'interest_rate', hue = 'loan_status', legend = True, kde = False, element = 'poly', bins = 25)
plt.xticks(rotation = 90);

In [ ]:
sns.histplot(data = train_clean, x = 'home_ownership', hue = 'loan_status', legend = True, kde = False, element = 'poly', bins = 15)
plt.xticks(rotation = 90);

**Observations**

There were no new observations from the above plots, other than the fact that we can confirm now there is very little colinearity between any of the features in the given dataset. Box plots were also made in a separate notebook that also yielded similar conclusions and will not be included here as many of the observations are similar.

**Scatter Plots**

Using scatter plots to try and see if there are any features that are similar to each other or may be related to each other that yield any patterns with the target column.

In [ ]:
fig = px.scatter(train_clean, x="loan_amount", y="total_collection_amount",
                color="loan_status", hover_data=['verification_status']);
fig.show()

In [ ]:
fig = px.scatter(train_clean, x="funded_amount", y="funded_amount_investor",
                color="loan_status", hover_data=['verification_status']);
fig.show()

In [ ]:
train_num['term'].unique()

In [ ]:
# treating term as if it were a categorical feature and geting the percetage defaults in each term
train_clean.groupby(['term'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

The 36 year term seems to have a higher percentage of defaults as compared to the 58 and 59 year terms.

In [ ]:
fig = px.scatter(train_clean, x="total_received_late_fee", y="collection_recovery_fee",
                color="loan_status", hover_data=['verification_status']);
fig.show()

In [ ]:
train_num['collection12months_medical'].unique()

In [ ]:
# treating collection12months_medical as if it were a categorical feature and geting the percetage defaults for each value
train_clean.groupby(['collection12months_medical'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

**Observations**

Again, the scatter plots did not yield many new findings, but that 2 of the continuous features can be treated as categorical features and can be inspected using the percentages of their values vs the target column.

**Feature Engineering**

Arithmatic Features:

In [ ]:
train_clean.head()

In [ ]:
# getting total funded amount
train_clean['loan_fund_total'] = train_clean['funded_amount'] + train_clean['funded_amount_investor']

# getting proportion loan funded
train_clean['prop_loan_funded'] = train_clean['loan_fund_total'] / train_clean['loan_amount']

Reducing the number of values in batch enrolled by grouping into 3 broad groups

In [ ]:
train_clean['batch_enrolled'].unique()

In [ ]:
train_clean.groupby(['batch_enrolled'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

In [ ]:
# Creating lists to store the unique values in each group
batch_group_1 = ['BAT2522922', 'BAT2136391', 'BAT2428731',
       'BAT5341619', 'BAT4694572', 'BAT4808022', 'BAT2558388',
       'BAT2078974', 'BAT2252229', 'BAT2333412', 'BAT5849876',
       'BAT2833642', 'BAT2803411', 'BAT5525466', 'BAT5714674',
       'BAT4722912', 'BAT3873588', 'BAT1780517',
       'BAT4271519', 'BAT5811547', 'BAT1184694', 'BAT4136152',
       'BAT3193689', 'BAT1467036', 'BAT2575549', 'BAT4351734',
       'BAT1104812', 'BAT5924421', 'BAT1930365', 'BAT1766061',
       'BAT5489674', 'BAT3865626', 'BAT5629144', 'BAT5547201',
       'BAT224923', 'BAT3726927', 'BAT3461431']

batch_group_2 = ['BAT1586599', 'BAT1761981', 'BAT2003848']

batch_group_3 = ['BAT1135695']

In [ ]:
# Define a dictionary mapping each list to its corresponding replacement string
batch_groups = {'batch_group_1': 'BAT 1', 'batch_group_2': 'BAT 2', 'batch_group_3': 'BAT 3'}

# Replace the values in the 'batch_enrolled' column based on membership in the original lists
train_clean['batch_enrolled'] = train_clean['batch_enrolled'].apply(lambda x: batch_groups['batch_group_1'] if x in batch_group_1
                                                                                    else (batch_groups['batch_group_2'] if x in batch_group_2
                                                                                    else batch_groups['batch_group_3']))



In [ ]:
# Check percentages for ne batch_enrolled column
train_clean.groupby(['batch_enrolled'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

Reducing the number of values in grades and sub grades by using the same method as the above

In [ ]:
train_clean.groupby(['grade'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

In [ ]:
# Creating lists to store the unique values in each group
grade_1 = ['A', 'B']
grade_2 = ['C', 'D', 'E']
grade_3 = ['F', 'G']

# Define a dictionary mapping each list to its corresponding replacement string
grade_groups = {'grade_1': 'grade1', 'grade_2': 'grade2', 'grade_3': 'grade3'}

# Replace the values in the 'grade' column based on membership in the original lists
train_clean['grade'] = train_clean['grade'].apply(lambda x: grade_groups['grade_1'] if x in grade_1
                                                                                    else (grade_groups['grade_2'] if x in grade_2
                                                                                    else grade_groups['grade_3']))

In [ ]:
# check percentages for train dataset grade column
train_clean.groupby(['grade'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

In [ ]:
# checking the list for sub-grade
train_clean.groupby(['sub_grade'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

In [ ]:
# split them into groups 7-8.99%, 9-9.99%, 10-11%, above 11%
sub_grade_1 = ['B2', 'B3', 'C1', 'D3', 'E2', 'E3', 'E5', 'F1', 'G2']
sub_grade_2 = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B4', 'B5', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D4', 'F3', 'F4', 'F5', 'G1','G4', 'G5']
sub_grade_3 = ['D5', 'E1', 'E4', 'F2']
sub_grade_4 = ['G3']

# Define a dictionary mapping each list to its corresponding replacement string
sub_grade_groups = {'sub_grade_1': 'subgrade1', 'sub_grade_2': 'subgrade2', 'sub_grade_3': 'subgrade3', 'sub_grade_4': 'subgrade4'}

# Replace the values in the 'sub_grade' column based on membership in the original lists
train_clean['sub_grade'] = train_clean['sub_grade'].apply(lambda x: sub_grade_groups['sub_grade_1'] if x in sub_grade_1
                                                                                    else (sub_grade_groups['sub_grade_2'] if x in sub_grade_2
                                                                                    else sub_grade_groups['sub_grade_3'] if x in sub_grade_3
                                                                                    else sub_grade_groups['sub_grade_4']))

In [ ]:
# check percentages for sub_grade
train_clean.groupby(['sub_grade'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

In [ ]:
train_clean.head()

In [ ]:
train_clean.shape

In [ ]:
# save version one of dataset to csv
train_clean.to_csv('train_model.csv', index=False)

**Data manipulation after first round of modelling**

Converting some of the numerical/continuous features to categorical ones by grouping them.

In [ ]:
train_clean.describe()

In [ ]:
cont_list = ['loan_amount', 'funded_amount', 'funded_amount_investor',
            'interest_rate', 'home_ownership', 'debitto_income',
            'inquires_sixmonths', 'open_account', 'public_record', 'revolving_balance', 'revolving_utilities', 'total_accounts',
            'total_received_interest', 'total_received_late_fee', 'recoveries', 'collection_recovery_fee',
            'lastweek_pay', 'total_collection_amount', 'total_current_balance', 'total_revolving_credit_limit', 'loan_fund_total',
            'prop_loan_funded']

In [ ]:
train_clean[cont_list].dtypes

In [ ]:
train_clean['loan_amount'].unique()

In [ ]:
# treating term as a catergorical feature
train_clean['term'] = train_clean['term'].astype(str)
train_clean['term'].dtypes

In [ ]:
# Checking %
train_clean.groupby(['term'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

In [ ]:
# treating term as a catergorical feature
train_clean['collection12months_medical'] = train_clean['collection12months_medical'].astype(str)
train_clean['collection12months_medical'].dtypes

In [ ]:
# Checking %
train_clean.groupby(['collection12months_medical'])['loan_status'].apply(lambda x: (x > 0).sum() / x.size * 100)

In [ ]:
train_clean['delinquency_twoyears'].astype(int)

In [ ]:
def bin_continuous_features(df, feature_list):
    for feature in feature_list:
        bins = pd.qcut(df[feature], q=4, duplicates='drop')
        df[feature] = bins
    return df

In [ ]:
train_clean = bin_continuous_features(train_clean, cont_list)

In [ ]:
train_clean.head()

In [ ]:
# saving dataset ready for modelling
train_clean.to_csv('train_model_a.csv', index=False)